PPCA lavora con più variabili osservabili anzichè una sola

In [39]:
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import extract_correct_csv
from scipy import stats
import pandas as pd
import warnings

from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior

In [40]:
warnings.simplefilter(action="ignore", category=FutureWarning)

In [41]:
scaler = StandardScaler()

# extract data social anxiety

In [42]:
prova_3_subj = [2, 4, 10]

In [43]:
global_e_labels = []
global_subject = []
for i in prova_3_subj:
    subj_ = extract_correct_csv.read_correct_subject_csv(i)
    csv_ = 'data/LookAtMe_0'+str(subj_)+'.csv'
    global_data = pd.read_csv(csv_, sep='\t')
    y = np.array(list([int(d>2) for d in global_data['rating']]))
    e_labels = y[:,np.newaxis]  # rating > 2
    global_e_labels = global_e_labels + e_labels.tolist()
    subject = np.array(list([s for s in global_data['subject']]))[:, np.newaxis]
    global_subject = global_subject + subject.tolist()
#e_labels = e_labels[num_trials_to_remove:]
'''N_e = e_labels.shape[0]
D_e = e_labels.shape[1]'''
global_e_labels = np.array(global_e_labels)
global_subject = np.array(global_subject)

N_e = global_e_labels.shape[0]
D_e = global_e_labels.shape[1]
print(N_e, D_e)

N_sub = global_subject.shape[0]
D_sub = global_subject.shape[1]
print(N_sub, D_sub)

480 1
480 1


In [44]:
gl = np.squeeze(global_subject)

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [45]:
global_subject_df = pd.DataFrame(global_subject, columns=['subject'])
global_subject_df.subject.unique()

array([ 2,  4, 10])

In [46]:
subject_dict = dict(zip(global_subject_df.subject.unique(), range(len(prova_3_subj))))
subj_def = global_subject_df.replace(subject_dict).values

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [47]:
show = False
# definition of the feature extractors to be used later
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(1,0))

# definition of the physiological signals to be extracted
eda_ = physio.EDA(f3)
hr_ = physio.HR(f3)
pupil_ = behavior.PUPIL(f3)

# extraction of the desired data from the dataset
elenco_subj = ([str(d) for d in prova_3_subj])
d = datasets.FEAR(signals={hr_,pupil_,eda_}, subjects=set(elenco_subj))

for s in d.signals:
    # preprocess ...
    if s.name =='EDA':
        s.preprocess(show=show,new_fps=500)
        s.feature_ext.extract_feat(s,show=show)
    else:
        if s.name == 'HR':
            list_hr_test = s.raw[0]['data']
            s.preprocess(show=show, useneurokit=True)
            s.feature_ext.extract_feat(s,show=show)

        else:
            s.feature_ext.extract_feat_without_preprocess(s, show=show)

    #add feature extraction for eda before preprocessing

    # ... and extract features from each signal type


for sig in d.signals:
    if sig.name=='EDA':
        eda_data = sig.features
    if sig.name=='HR':
        hr_data = sig.features
    if sig.name=='PUPIL':
        pupil_data = sig.features

>> Loading EDA for subject 2 and all sessions from dataset FEAR
>> Loading EDA for subject 10 and all sessions from dataset FEAR
>> Loading EDA for subject 4 and all sessions from dataset FEAR
>> Loading HR for subject 2 and all sessions from dataset FEAR
>> Loading HR for subject 10 and all sessions from dataset FEAR
>> Loading HR for subject 4 and all sessions from dataset FEAR
>> Loading PUPIL for subject 2 and all sessions from dataset FEAR
>> Loading PUPIL for subject 10 and all sessions from dataset FEAR
>> Loading PUPIL for subject 4 and all sessions from dataset FEAR
>> Processing EDA ...
>> Extracting mean features from EDA signal, adopting (1, 0) window ...
>> Processing HR ... using neurokit


/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._me

>> Extracting mean features from HR signal, adopting (1, 0) window ...
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...


In [48]:
TRIAL = 160*len(prova_3_subj)

In [49]:
hr = np.array(hr_data)
hr = hr.reshape((TRIAL, int(hr.shape[0]/TRIAL*hr.shape[1])))

In [50]:
pupil = np.array(pupil_data)
pupil = pupil.reshape((TRIAL, int(pupil.shape[0]/TRIAL*pupil.shape[1])))

In [51]:
eda = np.array(eda_data)
eda = eda.reshape((TRIAL,int(eda.shape[0]/TRIAL*eda.shape[1])))

In [52]:
hr.shape

(480, 6)

In [53]:
N_pupil = pupil.shape[0]
D_pupil = pupil.shape[1]

N_hr = hr.shape[0]
D_hr = hr.shape[1]

N_eda = eda.shape[0]
D_eda = eda.shape[1]
K = 1

print(N_pupil,D_pupil)
print(N_hr,D_hr)
print(N_eda,D_eda)
print(N_e, D_e)

480 5
480 6
480 6
480 1


In [54]:
hr.shape

(480, 6)

In [55]:
coords = {'subject': global_subject_df.subject.unique(), 'tot_trial':np.arange(N_hr)}

In [56]:
coords['subject']

array([ 2,  4, 10])

In [61]:
with pm.Model(coords=coords) as sPPCA:
    #dati osservabili
    hr_data = pm.MutableData("hr_data", hr.T)
    #pupil_data = pm.MutableData("pupil_data", pupil)
    #eda_data = pm.MutableData("eda_data", eda)

    #e_data = pm.ConstantData("e_data", global_e_labels)

    #matrici pesi
    mu_hr = pm.Normal('mu_hr',at.zeros([D_hr,K]), 10) # hyperprior 1
    sigma_hr = pm.Exponential('sigma_hr', 2*at.ones([D_hr,K]))# hyperprior 2
    Whr = pm.Normal('Whr', mu=mu_hr, sigma=sigma_hr, shape=[6,3])
    #Wpupil = pm.Normal('Wpupil', mu=at.zeros([D_pupil, K]), sigma=2.0 * at.ones([D_pupil, K]), shape=[D_pupil, K])

    #Weda = pm.Normal('Weda', mu=at.zeros([D_eda, K]), sigma=2.0 * at.ones([D_eda, K]), shape=[D_eda, K])

    #weight matrix for pain expectation.
    #check mu,sigma,shape
    #We = pm.Normal('W_e', mu=at.zeros([D_e, K]), sigma=2.0 * at.ones([D_e,K]), shape=[D_e, K])

    #latent space
    c = pm.Normal('c', mu=at.zeros([len(prova_3_subj),K]), sigma=at.ones([len(prova_3_subj),K]), shape=[len(prova_3_subj),K])
    #subject_idx = 2
    subject_idx = pm.MutableData("subject_idx", subj_def)

    # dati dell'hrv interpretati come una gaussiana
    # a_subjects_hr= pm.Normal("a_subjects_hr",mu=Whr.dot(c.T), sigma=10) #dims=[D_hr,N_hr]
    theta_hr = Whr[subject_idx]
    print(theta_hr.shape.eval())
    sigma2_hr = pm.Exponential("sigma2_hr",1.0)
    x_hr = pm.Normal('x_hr', mu=theta_hr.dot(c.T), sigma=sigma2_hr ,shape=[D_hr, N_hr], observed=hr_data)


    # dati della dilatazione pupille interpretati come una gaussiana
    '''a_pupil = pm.Normal('mu_pupil', Wpupil.dot(c.T), 10) # hyperprior 1
    sigma_pupil = pm.Exponential('sigma_pupil', 1)# hyperprior 2
    a_subjects_pupil = pm.Normal("a_subjects_pupil",mu=a_pupil, sigma=sigma_pupil) #dims=[D_hr,N_hr]
    theta_pupil = a_subjects_pupil[subject_idx]
    sigma2_pupil = pm.Exponential('sigma2_pupil', 1.0)# hyperprior 2
    x_pupil = pm.Normal('x_pupil', mu=theta_pupil, sigma=sigma2_pupil, shape=[D_pupil, N_pupil], observed=pupil_data)

    #eda
    a_eda = pm.Normal('mu_eda', Weda.dot(c.T), 10) # hyperprior 1
    sigma_eda = pm.Exponential('sigma_eda', 1) # hyperprior 2
    a_subjects_eda= pm.Normal("a_subjects_eda",mu=a_eda, sigma=sigma_eda) #dims=[D_hr,N_hr]
    theta_eda = a_subjects_eda[subject_idx]
    sigma2_eda = pm.Exponential('sigma2_eda', 1.0) # hyperprior 2
    x_eda = pm.Normal('x_eda', mu=theta_eda, sigma=sigma2_eda, shape=[D_eda,N_eda], observed=eda_data)'''

    # pain expectation. ciò che dovremmo inferire dato c
    # due strade: binary o multiclass (1-4)
    # p = probability of success?
    #x_e = pm.Bernoulli('x_e' , p=pm.math.sigmoid(We.dot(c.T)) , shape =[N_e,D_e], observed=e_data)

    #x_hr = pm.Bernoulli('x_hr', p=pm.math.sigmoid(Whr.dot(c.T)), shape=[D_hr, N_hr], observed=hr_data)
    #x_eda = pm.Bernoulli('x_eda', p=pm.math.sigmoid(Weda.dot(c.T)), shape=[D_eda, N_eda], observed=eda_data)

[480   1   3]


In [58]:
sPPCA.free_RVs

[mu_hr ~ N(f(), 10),
 sigma_hr ~ Exp(f()),
 Whr ~ N(mu_hr, sigma_hr),
 c ~ N(f(), f()),
 sigma2_hr ~ Exp(f())]

In [59]:
gv = pm.model_to_graphviz(sPPCA)
gv.view()

'.gv.pdf'

In [60]:
with sPPCA:
    approx = pm.fit(100000, callbacks=[pm.callbacks.CheckParametersConvergence(tolerance=1e-4)])
    trace = approx.sample(500)

'''with sPPCA:
    trace = pm.sample(1000,init='advi+adapt_diag',chains=1,target_accept=0.95)'''

qt5ct: using qt5ct plugin
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/16@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/16@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/22@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/24@2x/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24/"
Invalid Context= "Mimetypes" line for icon theme:  "/usr/share/icons/Mint-Y/mimetypes/24@2x/"
Invalid Context= "Apps" line for icon theme:  "/usr/share/icons/Mint-Y/apps/32/"
Invalid Context= "Apps" line

ValueError: ('shapes (480,3) and (1,3) not aligned: 3 (dim 1) != 1 (dim 0)', (480, 3), (1, 3))
Apply node that caused the error: Dot22(Reshape{2}.0, InplaceDimShuffle{x,0}.0)
Toposort index: 136
Inputs types: [TensorType(float64, (None, None)), TensorType(float64, (1, None))]
Inputs shapes: [(480, 3), (1, 3)]
Inputs strides: [(24, 8), (24, 8)]
Inputs values: ['not shown', array([[ 1.03019583, -0.99224577, -1.69809405]])]
Outputs clients: [[Reshape{3}(Dot22.0, Join.0)]]

Backtrace when the node is created (use Aesara flag traceback__limit=N to make it longer):
  File "/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/ipykernel/zmqshell.py", line 528, in run_cell
    return super().run_cell(*args, **kwargs)
  File "/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2885, in run_cell
    result = self._run_cell(
  File "/home/paolo/matteo/matteo/unimi/tesi_master/code/my_env/lib/pyth

In [ ]:
#az.plot_posterior(mean_field.sample(1000), color="LightSeaGreen")
az.plot_trace(trace)
plt.show()

In [ ]:
with sPPCA:
    # update values of predictors:
    # pm.set_data({"hr_data":hr})
    # use the updated values and predict outcomes and probabilities:
    posterior_predictive = pm.sample_posterior_predictive(
        trace, random_seed=123)

In [ ]:
e_pred = posterior_predictive.posterior_predictive["x_e"]

In [ ]:
e_pred_mode = np.squeeze(stats.mode(e_pred[0], keepdims=False)[0])[:,np.newaxis]

In [ ]:
train_accuracy_exp = accuracy_score(e_labels, e_pred_mode)
print('\n\tTrain Accuracy Pain Expectation: ' + str(train_accuracy_exp))
print(' ')

--------------------------------------------------------------------------------------------